# At https://www.kaggle.com/datasets/smartfactoryowl/metalclass you will find the materials.csv file with the corresponding licence.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Read data
path = 'Hackathon/materials.csv'
# set row with index 1 as header and set column with the name 'channel' as index of the DataFrame
df = pd.read_csv(path, header = 1, index_col='channel')
df

,Al_1,Al_2,Al_3,Al_4,Al_5,Asilikos,Batterien,Cu_0,Cu_1,Cu_2,...,Cu_4,Cu_5,E-Schrott,Erdreich_1,Erdreich_2,Erz,Melamine,PVC,Stuckgips,Zement
channel,,,,,,,,,,,,,,,,,,,,,
1,114029.0,0.0,120460.0,139299.0,126009.0,129635.0,245381.0,9344.0,632245.0,521859.0,...,668144.0,365089.0,502561.0,208509.0,214481.0,32772.0,0.0,74515.0,110722.0,510988.0
2,127755.0,0.0,133759.0,155997.0,140620.0,142635.0,244500.0,10683.0,633533.0,522030.0,...,671510.0,365492.0,500074.0,208043.0,214120.0,36649.0,0.0,86027.0,109931.0,507617.0
3,131648.0,0.0,137743.0,160507.0,144329.0,146872.0,244015.0,10888.0,632122.0,520776.0,...,672515.0,365888.0,499030.0,206711.0,213329.0,38136.0,0.0,88909.0,109100.0,505638.0
4,131825.0,0.0,138640.0,160614.0,144185.0,146709.0,242481.0,10956.0,633030.0,521682.0,...,671945.0,365529.0,496066.0,204582.0,211625.0,38590.0,0.0,89854.0,108346.0,502495.0
5,131731.0,1.0,138780.0,160435.0,143687.0,145903.0,241698.0,10795.0,631888.0,520643.0,...,670891.0,365304.0,492185.0,204442.0,209939.0,38478.0,0.0,90026.0,107251.0,498474.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16240,0.0,0.0,2.0,0.0,3.0,2.0,1.0,0.0,16.0,19.0,...,25.0,5.0,10.0,3.0,1.0,0.0,0.0,0.0,4.0,11.0
16241,0.0,2.0,1.0,1.0,1.0,0.0,5.0,0.0,19.0,14.0,...,25.0,8.0,11.0,4.0,1.0,2.0,0.0,3.0,1.0,12.0
16242,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,21.0,6.0,...,16.0,3.0,10.0,3.0,1.0,2.0,0.0,3.0,2.0,9.0


In [ ]:
# Data Genaration by sampling
def sample_df(data, n):
    """ Creates a DataFrame object containing the counts of each element 
    returned from a sample with replacement for each column of the given DataFrame.

    Parameters
    ----------
    data: pandas DataFrame
        DataFrame to be sampled from.
    n : int
        Number of values sampled from each column.

    Returns
    -------
    pandas DataFrame
        A new DataFrame with the same number of rows and columns as the original.
    """

    # Create a DataFrame that has the same column names and index as data and is empty
    # besides the Energy (index) column. 
    sample_table = pd.DataFrame(columns = data.columns, index = data.index)             

    # Calculate the probability distribution of each column of the original dataset
    pvals = data / data.sum()

    # Iterating over every column and using the numpy multinomial function with the
    # probability distribution of the data DataFrame as weights to generate a sample
    # of each alloy with replacement. The sampled values are saved in the corresponding
    # column in the sample_table that is returned as the functions return value.
    for k in sample_table.columns:
        sample_table[k] = np.random.multinomial(n=n, pvals=pvals[k])
    
    return sample_table


In [ ]:
# Preparation for data classification
def maxloglike(longtime_spectra):
    s = longtime_spectra.sum()
    
    norm = (longtime_spectra + 1) / s

    norm_log = np.log(norm)
    norm_log[np.isinf(norm_log)] = 0
    
    return norm_log

In [ ]:
# Matrix for data classification
A = [maxloglike(df[i]) for i in df.columns]

In [ ]:
# Simulate a short measurement by sampling the data with a small amount of counts
test_set = sample_df(df,2000)
test_set

,Al_1,Al_2,Al_3,Al_4,Al_5,Asilikos,Batterien,Cu_0,Cu_1,Cu_2,...,Cu_4,Cu_5,E-Schrott,Erdreich_1,Erdreich_2,Erz,Melamine,PVC,Stuckgips,Zement
channel,,,,,,,,,,,,,,,,,,,,,
1,50,0,22,38,31,37,25,35,34,32,...,22,28,25,53,55,53,0,30,36,40
2,32,0,28,35,42,44,38,30,30,26,...,37,22,33,46,38,46,0,25,49,33
3,58,0,29,49,34,40,25,22,33,32,...,27,35,30,36,43,42,0,29,46,44
4,29,0,45,60,46,29,33,25,38,39,...,25,35,37,39,42,28,0,38,28,42
5,34,0,37,54,37,46,25,38,30,36,...,25,28,30,45,40,41,0,38,39,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16240,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16241,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16242,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Classification of sampled test data
for k in test_set.columns:
    print('test_data: ',k)
    print('prediction:',test_set.columns[np.argmax(A @ test_set[k])])
    print('__')

test_data:  Al_1
prediction: Al_1
__
test_data:  Al_2
prediction: Al_2
__
test_data:  Al_3
prediction: Al_3
__
test_data:  Al_4
prediction: Al_4
__
test_data:  Al_5
prediction: Al_5
__
test_data:  Asilikos
prediction: Asilikos
__
test_data:  Batterien
prediction: Batterien
__
test_data:  Cu_0
prediction: Cu_0
__
test_data:  Cu_1
prediction: Cu_1
__
test_data:  Cu_2
prediction: Cu_1
__
test_data:  Cu_3
prediction: Cu_3
__
test_data:  Cu_4
prediction: Cu_4
__
test_data:  Cu_5
prediction: Cu_5
__
test_data:  E-Schrott
prediction: E-Schrott
__
test_data:  Erdreich_1
prediction: Erdreich_1
__
test_data:  Erdreich_2
prediction: Erdreich_2
__
test_data:  Erz
prediction: Erz
__
test_data:  Melamine
prediction: Melamine
__
test_data:  PVC
prediction: PVC
__
test_data:  Stuckgips
prediction: Stuckgips
__
test_data:  Zement
prediction: Zement
__
